# Build Vector

In [2]:
import pandas as pd
import json
import requests

from datetime import datetime
from bs4 import BeautifulSoup

In [3]:
df = pd.read_json('meli_challenge/items.jl', lines=True)

In [4]:
df.head()

,link,appearences
0,http://www.iana.org/domains/reserved,0
1,http://www.iana.org/,1
2,http://www.iana.org/about/excellence,0
3,https://pti.icann.org/,3
4,http://www.iana.org/reviews,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   link         50 non-null     object
 1   appearences  50 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 928.0+ bytes


## Creating Vector for each link

In [6]:
# Load json lines to a dict-list

rows = []

with open('meli_challenge/items.jl') as f:

    for line in f:

        rows.append(json.loads(line))

In [7]:
i = 0

for row in rows:
    
    print(f'Processed link {i} of {len(rows)-1}')
    
    # Make a GET request to fetch the raw HTML content
    response = requests.get(row['link'])
    
    html_content = response.text

    # Parse the html content
    soup = BeautifulSoup(html_content, "lxml")
    
    # Vector #1 - URL character Length
    row['qty_character_url'] = len(row['link'])

    # Vector #2: Page Title Character Length
    row['qty_character_page_title'] = len(soup.html.head.title.text)
    
    # Vector #3: Page href http(s) qty
    row['qty_page_http_links'] = len([t for t in soup.find_all('a') if 'http' in t.get('href',[]) ])
    
    # Vector #4: Page internal links qty
    row['qty_page_internal_links'] = len([t for t in soup.find_all('a')  if 'http' not in t.get('href',[])])
    
    # Vector #5: Qty a html tag
    row['qty_total_a'] = len(soup.find_all('a'))
    
    # Vector #6: Qty <table>
    row['qty_total_table'] = len(soup.find_all("table"))
    
    # Vector #7: Qty tr
    row['qty_total_table_tr'] = len(soup.find_all("tr"))
    
    # Vector #8: Qty table td
    row['qty_total_table_td'] = len(soup.find_all("td"))
    
    # Vector #9: Response Size
    row['qty_bytes_response'] = len(response.content)
    
    # Vector #10: Qty Page H1
    row['qty_h1_page'] = len(soup.find_all("h1"))
    
    i += 1
    
df_enrichment = pd.DataFrame(rows)

Processed link 0 of 49
Processed link 1 of 49
Processed link 2 of 49
Processed link 3 of 49
Processed link 4 of 49
Processed link 5 of 49
Processed link 6 of 49
Processed link 7 of 49
Processed link 8 of 49
Processed link 9 of 49
Processed link 10 of 49
Processed link 11 of 49
Processed link 12 of 49
Processed link 13 of 49
Processed link 14 of 49
Processed link 15 of 49
Processed link 16 of 49
Processed link 17 of 49
Processed link 18 of 49
Processed link 19 of 49
Processed link 20 of 49
Processed link 21 of 49
Processed link 22 of 49
Processed link 23 of 49
Processed link 24 of 49
Processed link 25 of 49
Processed link 26 of 49
Processed link 27 of 49
Processed link 28 of 49
Processed link 29 of 49
Processed link 30 of 49
Processed link 31 of 49
Processed link 32 of 49
Processed link 33 of 49
Processed link 34 of 49
Processed link 35 of 49
Processed link 36 of 49
Processed link 37 of 49
Processed link 38 of 49
Processed link 39 of 49
Processed link 40 of 49
Processed link 41 of 49
Pr

In [9]:
df_enrichment.head(10)

,link,appearences,qty_character_url,qty_character_page_title,qty_page_http_links,qty_page_internal_links,qty_total_a,qty_total_table,qty_total_table_tr,qty_total_table_td,qty_bytes_response,qty_h1_page
0,http://www.iana.org/domains/reserved,0,36,36,7,61,68,2,16,52,10350,1
1,http://www.iana.org/,1,20,35,5,31,36,1,4,8,6735,1
2,http://www.iana.org/about/excellence,0,36,49,6,33,39,1,4,8,6505,1
3,https://pti.icann.org/,3,22,34,34,25,59,0,0,0,19736,5
4,http://www.iana.org/reviews,0,27,14,5,42,47,2,14,17,8456,1
5,http://www.iana.org/performance,0,31,28,6,50,56,4,14,22,8424,1
6,https://www.icann.org/privacy/tos,46,33,24,70,550,620,0,0,0,126112,1
7,http://www.iana.org/reports,0,27,14,6,1588,1594,2,1561,3120,286384,1
8,http://www.iana.org/about/presentations,0,39,20,4,161,165,2,133,520,36432,1
9,https://www.icann.org/privacy/policy,45,36,22,70,552,622,0,0,0,120167,1
